<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#WGCNA-run-documention" data-toc-modified-id="WGCNA-run-documention-1">WGCNA run documention</a></span></li><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-2">Dependencies</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Add-entrez-ID-and-run-GO" data-toc-modified-id="Add-entrez-ID-and-run-GO-3.0.1">Add entrez ID and run GO</a></span></li></ul></li></ul></li><li><span><a href="#Deprecated" data-toc-modified-id="Deprecated-4">Deprecated</a></span><ul class="toc-item"><li><span><a href="#Consensus-network" data-toc-modified-id="Consensus-network-4.1">Consensus network</a></span></li><li><span><a href="#Consensus-input" data-toc-modified-id="Consensus-input-4.2">Consensus input</a></span></li><li><span><a href="#Run-blockwise-consensus-function" data-toc-modified-id="Run-blockwise-consensus-function-4.3">Run blockwise consensus function</a></span></li></ul></li><li><span><a href="#Try-step-by-step-module-identification" data-toc-modified-id="Try-step-by-step-module-identification-5">Try step-by-step module identification</a></span></li></ul></div>

# WGCNA run documention

In [1]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INDICATE/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INDICATE/Plots/"
mountpath <- "/Users/anabbi/Desktop/H4H/INDICATE/wgcna/th/"

on samwise, open an R console:

In [4]:
log2(0.1)

[1] -3.321928

In [ ]:
clinical <- read.csv("~/immpedcan/wgcna/treehouse_metadata.csv", stringsAsFactors = F, header = T, row.names = 1)

#load IND tpm matrix. Copied from local to samwise
IND_mat <- read.csv(file = "~/immpedcan/wgcna/INDICATE.tpm_hg38_ENSG_HUGO.csv", header = T, stringsAsFactors = F,check.names = F)

#loading th data takes some time
tpm_mat <- read.table("~/pughlab/external_data/treehousegenomics/TumorCompendium_v11_PolyA_hugo_log2tpm_58581genes_2020-04-09.tsv", sep = "\t", header = T, stringsAsFactors = F, check.names = F)
rownames(tpm_mat) <- tpm_mat$Gene
tpm_mat$Gene <- NULL

#Subset to my cancer types
cancertypes <- c("Lymphoma","NBL","OS","RMS","SARC","WILMS", "EWS")
clinical_subset <- clinical[clinical$diseasetype %in% cancertypes,]

th_mat_subset <- tpm_mat[,colnames(tpm_mat) %in% clinical_subset$th_sampleid]
#Only those genes in IND matrix
th_mat_subset <- th_mat_subset[rownames(th_mat_subset) %in% IND_mat$HGNC_Symbol,]
# 24435 genes overlapp with IND matrix

#If log2(tpm) == 0 in more than half of samples, remove them
keepgenes <- rowSums(th_mat_subset == 0) <= ncol(th_mat_subset)/2
th_mat_subset <- th_mat_subset[keepgenes,]
dim(th_mat_subset)

#19014 genes overlap with IND and are > 0 in more than half of all samples (n = 835)

for (c in cancertypes){
    ids <- clinical_subset$th_sampleid[clinical_subset$diseasetype == c]
    cancer_mat <- th_mat_subset[,colnames(th_mat_subset) %in% ids]
    rownames(cancer_mat) <- rownames(th_mat_subset)
    print(c)
    print(dim(cancer_mat))
    write.csv(cancer_mat, file = paste0("~/immpedcan/wgcna/tpm_th_",c, ".csv"), quote = F)
}
"Lymphoma"
19014    25
"NBL"
19014   201
"OS"
19014   184
"RMS"
19014   168
"SARC"
19014    40
"WILMS"
19014   136
"EWS"
19014    81

quit()

Transfer to H4H

In [ ]:
ssh anabbi@xfer
cd immpedcan/wgcna/
scp *.csv anabbi@h4huhndata1:~/INDICATE/wgcna/th

run consensus_input.R on h4h

In [ ]:
cd INDICATE/wgcna/th
module load R/3.6.1
sbatch -t 5-00:00:00 --mem=120G -p veryhimem -c 12 -N 1 Rscript consensus_input_th.R

In [ ]:
mv slurm-2260326.out consensus_input_tpm_run.out

run sftthreshold.R on h4h

In [ ]:
sbatch -t 5-00:00:00 --mem=120G -p veryhimem -c 12 -N 1 Rscript sftthreshold.R

In [ ]:
mv slurm-2260333.out sftthreshold_signed_th.out

Soft power is 20.

Run auto mode:

In [ ]:
sbatch -t 5-00:00:00 --mem=900G -p superhimem -c 12 -N 1 Rscript consensus_wgcna_auto_p20_th.R 

And manual mode with minimum p for each cancer type with model fit > 0.8

In [ ]:
sbatch -t 5-00:00:00 --mem=900G -p superhimem -c 12 -N 1 Rscript consensus_manual_signed_th.R

# Dependencies

In [1]:
options(scipen = 999)

In [2]:
library(WGCNA)
library(biomaRt)

Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




# Paths

In [3]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INDICATE/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INDICATE/Plots/"
mountpath <- "/Users/anabbi/Desktop/H4H/INDICATE/wgcna/th/"

I copied this file dataInput_Treehouse_tpm.RData to local because it is big and doesnt read from mounted h4h

In [8]:
load(file = paste0(datapath, "WGCNA/TH/Consensus-dataInput_Treehouse_tpm.RData"))

In [9]:
load(file = paste0(mountpath, "Consensus_Network_TH_manual_signed_20.RData"))

In [12]:
table(moduleColors)

moduleColors
        black         brown          cyan     darkgreen      darkgrey 
          332           228            85            44            33 
   darkorange darkturquoise          grey        grey60    lightgreen 
           28            43         15966           535            51 
          red        salmon       skyblue         white        yellow 
         1162            58            24            28           393 

In [13]:
gene_module <- data.frame(moduleLabel = moduleLabels, moduleColor = moduleColors,
                          row.names = colnames(treehouse_gene_mat[[1]]$data))

In [14]:
head(gene_module)

,moduleLabel,moduleColor
,<dbl>,<chr>
A1BG,0,grey
A1BG-AS1,0,grey
A1CF,0,grey
A2M,22,darkgreen
A2M-AS1,0,grey
A2ML1,0,grey


In [15]:
ENSG_HGNCfile <- read.table(paste0(datapath, "exp_mat/EnsGeneID_Hugo_Observed_Conversions_TH.txt"), sep = "\t",
                           header = T, stringsAsFactors = F, check.names = F)

In [16]:
head(ENSG_HGNCfile)

,HugoID,EnsGeneID
,<chr>,<chr>
1,TSPAN6,ENSG00000000003.14
2,TNMD,ENSG00000000005.5
3,DPM1,ENSG00000000419.12
4,SCYL3,ENSG00000000457.13
5,C1orf112,ENSG00000000460.16
6,FGR,ENSG00000000938.12


In [17]:
THhugos <- rownames(gene_module)

In [18]:
hugo2ensg <- match(THhugos, ENSG_HGNCfile$HugoID)

gene_module$ensembl_id_TH <- ENSG_HGNCfile$EnsGeneID[hugo2ensg]

gene_module$ensembl_id <- gsub("[.].*", "", gene_module$ensembl_id_TH)

In [19]:
head(gene_module)

,moduleLabel,moduleColor,ensembl_id_TH,ensembl_id
,<dbl>,<chr>,<chr>,<chr>
A1BG,0,grey,ENSG00000121410.11,ENSG00000121410
A1BG-AS1,0,grey,ENSG00000268895.5,ENSG00000268895
A1CF,0,grey,ENSG00000148584.14,ENSG00000148584
A2M,22,darkgreen,ENSG00000175899.14,ENSG00000175899
A2M-AS1,0,grey,ENSG00000245105.2,ENSG00000245105
A2ML1,0,grey,ENSG00000166535.19,ENSG00000166535


### Add entrez ID and run GO

In [20]:
hg38 <- useMart(biomart="ENSEMBL_MART_ENSEMBL", host="useast.ensembl.org", 
                 path="/biomart/martservice", dataset="hsapiens_gene_ensembl")

In [21]:
TH_ensembl <- gene_module$ensembl_id

In [22]:
ensembl_hgnc_entrez <- getBM(filters="ensembl_gene_id",
                      attributes=c("ensembl_gene_id", "hgnc_symbol", "entrezgene_id", "gene_biotype"), 
                      values= TH_ensembl, mart=hg38)

In [23]:
head(ensembl_hgnc_entrez)

,ensembl_gene_id,hgnc_symbol,entrezgene_id,gene_biotype
,<chr>,<chr>,<int>,<chr>
1,ENSG00000000419,DPM1,8813,protein_coding
2,ENSG00000000460,C1orf112,55732,protein_coding
3,ENSG00000000971,CFH,3075,protein_coding
4,ENSG00000001626,CFTR,1080,protein_coding
5,ENSG00000001629,ANKIB1,54467,protein_coding
6,ENSG00000001630,CYP51A1,1595,protein_coding


In [24]:
hugo2ensg <- match(THhugos, ENSG_HGNCfile$HugoID)
gene_module$ensembl_id_TH <- ENSG_HGNCfile$EnsGeneID[hugo2ensg]
gene_module$ensembl_id <- gsub("[.].*", "", gene_module$ensembl_id_TH)

In [25]:
ensembl2entrez <- match(TH_ensembl, ensembl_hgnc_entrez$ensembl_gene_id)

gene_module$entrez <- ensembl_hgnc_entrez$entrezgene_id[ensembl2entrez]

In [26]:
write.csv(gene_module, file = paste0(datapath, "WGCNA/TH/gene_module_treehouse_manual_signed_20.csv"))

In [27]:
allentrez <- ensembl_hgnc_entrez$entrezgene_id[ensembl2entrez]

In [28]:
GOenr <- GOenrichmentAnalysis(gene_module$moduleColor, allentrez, organism = "human", nBestP = 10)

Warning message in GOenrichmentAnalysis(gene_module$moduleColor, allentrez, organism = "human", :
“This function is deprecated and will be removed in the near future. 
We suggest using the replacement function enrichmentAnalysis 
in R package anRichment, available from the following URL:
https://labs.genetics.ucla.edu/horvath/htdocs/CoexpressionNetwork/GeneAnnotation/”
Loading required package: org.Hs.eg.db

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, app

 GOenrichmentAnalysis: loading annotation data...
  ..of the 18175  Entrez identifiers submitted, 16720 are mapped in current GO categories.
  ..will use 16720 background genes for enrichment calculations.
  ..preparing term lists (this may take a while).. 
  ..working on label set 1 ..
    ..calculating enrichments (this may also take a while)..
    ..putting together terms with highest enrichment significance..


In [29]:
tab <- GOenr$bestPTerms[[4]]$enrichment

In [30]:
keepCols <- c(1, 2, 5, 6, 7, 12, 13)
screenTab <- tab[, keepCols]
numCols <- c(3, 4)
screenTab[, numCols] <- signif(apply(screenTab[, numCols], 2, as.numeric), 2)

In [31]:
write.csv(screenTab, file = paste0(datapath, "WGCNA/TH/GO_TH_cons_manual_signed_20.csv"))